In [1]:
%%capture
!pip install fairseq

In [2]:
!wget https://dl.fbaipublicfiles.com/fairseq/wav2vec/wav2vec_small_960h.pt

--2022-05-04 23:43:22--  https://dl.fbaipublicfiles.com/fairseq/wav2vec/wav2vec_small_960h.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 944029831 (900M) [application/octet-stream]
Saving to: ‘wav2vec_small_960h.pt’

wav2vec_small_960h. 100%[===================>] 900.30M  45.9MB/s    in 19s     

2022-05-04 23:43:41 (47.5 MB/s) - ‘wav2vec_small_960h.pt’ saved [944029831/944029831]



In [3]:
#!mkdir config
#!wget https://raw.githubusercontent.com/pytorch/fairseq/main/examples/wav2vec/config/finetuning/base_960h.yaml -O config/base_960h.yaml 

In [4]:
!mkdir config
%%writefile config/base_960h.yaml
common:
  fp16: true
  log_format: json
  log_interval: 200

checkpoint:
  no_epoch_checkpoints: true
  best_checkpoint_metric: wer
  restore_file: wav2vec_small_960h.pt

task:
  _name: audio_finetuning
  data: ???
  normalize: false
  labels: ltr

dataset:
  num_workers: 1
  max_tokens: 3200000
  skip_invalid_size_inputs_valid_test: true
  valid_subset: dev_other

distributed_training:
  ddp_backend: legacy_ddp
  distributed_world_size: 1

criterion:
  _name: ctc
  zero_infinity: true

optimization:
  max_update: 360000
  lr: [0.0001]
  sentence_avg: true

optimizer:
  _name: adam
  adam_betas: (0.9,0.98)
  adam_eps: 1e-08

lr_scheduler:
  _name: tri_stage
  phase_ratio: [0.1, 0.4, 0.5]
  final_lr_scale: 0.05

model:
  _name: wav2vec_ctc
  w2v_path: ???
  apply_mask: true
  mask_prob: 0.5
  mask_channel_prob: 0.1
  mask_channel_length: 64
  layerdrop: 0.1
  activation_dropout: 0.1
  feature_grad_mult: 0.0
  freeze_finetune_updates: 0

SyntaxError: invalid syntax (945188718.py, line 3)

In [ ]:
!fairseq-hydra-train task.data=../input/create-ljspeech-splits/10mins/ model.w2v_path=wav2vec_small_960h.pt --config-dir config --config-name base_960h